In [1]:
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
# from keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import matplotlib


In [2]:
%cd /Users/Kunal/Projects/TCH_CardiacSignals_F20/

/Users/kunal/Projects/TCH_CardiacSignals_F20


In [3]:
data = np.load(os.path.join("Working_Data", "Fixed_Dim_HBs_Idx" + "11" + ".npy"))
data = np.load(os.path.join("Working_Data", "Fixed_Dim_HBs_Idx" + "1" + ".npy"))
lead_1 = data[:,:,0]
lead_2 = data[:,:,1]
lead_3 = data[:,:,2]
lead_4 = data[:,:,3]

In [4]:
batch_size = 1
dataset = tf.data.Dataset.from_tensor_slices(lead_1).repeat().batch(batch_size)

In [5]:
iterator = iter(dataset)
features = next(iterator).numpy()

n_input = 100
n_output = n_input

# encoder
n_hidden1 = 75
n_hidden2 = 35

# decoder
n_hidden3 = n_hidden1
lr = 0.01
l2_reg = 0.0001

In [7]:
xav_init = tf.keras.initializers.GlorotUniform()
l2_regularizer = tf.keras.regularizers.L2(l2_reg)

In [19]:
from functools import partial
dense_layer = partial(keras.layers.Dense,activation="relu",kernel_initializer=xav_init,kernel_regularizer=l2_regularizer)
h1 = dense_layer(n_hidden1)(features)
h2 = dense_layer(n_hidden2)(h1)
h3 = dense_layer(n_hidden1)(h2)
outputs = dense_layer(n_output, activation = None)(h3)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



In [22]:
loss = tf.reduce_mean(np.square(outputs - features))
optimizer = keras.optimizers.Adam(learning_rate=lr)
train = optimizer.minimize(loss, var_list =None)



ValueError: Passed in object of type <class 'NoneType'>, not tf.Tensor